RBM用于协同过滤算法介绍
===

Ruslan Salakhutdinov首次提出了使用RBM求解Netflix竞赛中的协同过滤问题。其首先假设协同过滤系统中共有M个用户以及N个项目，用户对于某一个项目的偏好由评分表示（1分—K分，分别表示从不喜欢到喜欢的程度），因此可获得（稀疏性很高）的用户—项目评分矩阵。矩阵中在第m行第n列位置上的元素表示用户m对于项目n的评分值。RuslanSalakhutdinov对传统的受限玻尔兹曼机做了两点改变以适合处理协同过滤问题。第一，可见单元使用Softmax神经元；第二，某个用户可能只对M部电影中的一部分电影进行过评分，而对于没有评分的记录使用一种特殊的神经元表示，这种神经元不与任何隐单元连接。其具体结构如下：
![](rbmcf.png)
从图中可以看出，Softmax神经元为一个长度为K的向量，并且这个向量每次只有一个分量为1，其余分量则为0。而评分缺失的Softmax神经元可以同样视为一个长度为K的向量，只不过其所有的分量始终为0。这样某用户的评分可以用的矩阵V来表示，表示矩阵V第k行第i列的元素，表示该用户对于电影i的评分为k，在给定可见单元状态，隐单元的激活概率变为：
$$P(h_j=1|V)={1 \over 1+exp(-b_j-\sum_{i=1}^M\sum_{k=1}^Kv_i^kw_{ij}^k)} \qquad (1)$$
同理，在给定隐单元状态，可见单元状态的激活概率变为：
$$P(v_i^k=1|h)={exp(a_i^k+\sum_{j=1}^Fw_{ij}^kh_j) \over
\sum_{i=1}^Kexp(a_i^l+\sum_{j=1}^Fw_{ij}^lh_j)} \qquad (2)$$
使用CD算法，各参数的更新法则为:
$$\Delta W_{ij}^k= \epsilon (<v_i^kh_j)^+ - <v_i^kh_j>^T) $$
$$\Delta a_{i}^k= \epsilon (<v_i^k)^+ - <v_i^k>^T) $$
$$\Delta b_{j}^k= \epsilon (<h_j)^+ - <h_j>^T) $$

推荐阶段：以上为基于受限玻尔兹曼机的学习阶段，可以获得整个网络的全部参数。而在其推荐阶段的过程如下：

1. 输入为某一个用户u和某一个项目i，希望的目标输出是用户u对用户i的预测评分；
2. 将此用户u的所有评分作为RBM的softmax单元的输入；
3. 根据式（1）计算所有隐单元j 的激活概率；
4. 根据式（2）为项目i计算每一个评分k(k=1,…,K)所对应的激活概率；
6. 预测用户u对项目i的评分，取预测的最大值或者取所有概率的期望。